# Usando Modelos
Neste notebook vamos mostrar como fazer uso de um modelo, mas antes disso vamos primeiro carregar o nosso conjunto de dados e ajustar os tipos científicos deste.

In [1]:
using Queryverse
using MLJ

In [3]:
dados = Queryverse.load("iris.csv") |> DataFrame
first(dados, 3)

,sepallength,sepalwidth,petallength,petalwidth,variety
,Float64,Float64,Float64,Float64,String
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3.0,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa


In [4]:
coerce!(dados, autotype(dados))
schema(dados)

┌─────────────┬───────────────────────────┬───────────────┐
│ _.names     │ _.types                   │ _.scitypes    │
├─────────────┼───────────────────────────┼───────────────┤
│ sepallength │ Float64                   │ Continuous    │
│ sepalwidth  │ Float64                   │ Continuous    │
│ petallength │ Float64                   │ Continuous    │
│ petalwidth  │ Float64                   │ Continuous    │
│ variety     │ CategoricalString{UInt32} │ Multiclass{3} │
└─────────────┴───────────────────────────┴───────────────┘
_.nrows = 150


Agora que o nosso conjunto está com os tipos adequados, vamos para a escolha do modelo.

In [11]:
y, x = unpack(dados, ==(:variety), colnames -> true)
for m in models(matching(x,y))
    if m.prediction_type == :probabilistic &&
        m.is_pure_julia == true
        println(rpad(m.name, 28), m.package_name)
    end
end

AdaBoostStumpClassifier     DecisionTree
BayesianLDA                 MultivariateStats
BayesianSubspaceLDA         MultivariateStats
ConstantClassifier          MLJModels
DecisionTreeClassifier      DecisionTree
EvoTreeClassifier           EvoTrees
GaussianNBClassifier        NaiveBayes
KNNClassifier               NearestNeighbors
LDA                         MultivariateStats
LinearBinaryClassifier      GLM
LogisticClassifier          MLJLinearModels
MultinomialClassifier       MLJLinearModels
RandomForestClassifier      DecisionTree
SubspaceLDA                 MultivariateStats


Vamos então utilizar o __DecisionTreeClassifier__ do pacote __DecisionTree__.

In [13]:
dtc = @load DecisionTreeClassifier pkg="DecisionTree"

DecisionTreeClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0,
    pdf_smoothing = 0.0,
    display_depth = 5) @ 1…28

Já temos o nosso modelo escolhido e temos acesso aos seus parâmetros. O que faremos agora é criar uma machine. A __machine__ é uma estrutura que encapsula tanto o modelo quanto os dados e pode conter informações do modelo treinado, ela não faz o ajuste do modelo por si só. Entretanto, ela verifica se os tipos científicos são compatíveis com o modelo e avisa caso não sejam. 

In [14]:
arvore = machine(dtc, x, y)

Machine{DecisionTreeClassifier} @ 1…97


A __machine__ é utilizada para modelos supervisionados e não supervisionados, faremos exemplo de uso desta para os dois casos.

### Modelo supervisionado

Já temos carregado um modelo do tipo supervisionado, vamos começar fazendo uma divisão dos nossos dados em dados de treino e dados de teste.

In [18]:
treino, teste = partition(eachindex(y), 0.7, shuffle = true);

Agora vamos ajustar os parâmetros do nosso modelo com base nos nossos dados de treino (informaremos as linhas), para isso vamos usara função __fit!__ usando a nossa __machine__.

In [19]:
fit!(arvore, rows=treino)

┌ Info: Training Machine{DecisionTreeClassifier} @ 1…97.
└ @ MLJBase /home/phelipe/.julia/packages/MLJBase/uJ1jK/src/machines.jl:182


Machine{DecisionTreeClassifier} @ 1…97


A função __fit!__ modificou os parâmetros do nosso modelo, podemos inspecionar os resultados com o uso da função __fitted_params__ e nossa __machine__.

In [29]:
fitted_params(arvore)

(tree = Decision Tree
Leaves: 7
Depth:  4,
 encoding = Dict{CategoricalString{UInt32},UInt32}("Versicolor" => 0x00000002,"Setosa" => 0x00000001,"Virginica" => 0x00000003),)

O retorno da função __fitted_params__ varia de modelo para modelo, normalmente ele retorna uma tupla onde o primeiro elemento são os parâmetros ajustados do modelo e o segundo apresenta a forma como as classes foram nomeadas.

É possível agora utilizar a __machine__ para fazer predições com a função __predict__.

In [41]:
y_pred = predict(arvore, rows = teste)
@show y_pred[1]

y_pred[1] = UnivariateFinite(Setosa=>0.0, Versicolor=>0.0, Virginica=>1.0)


UnivariateFinite(Setosa=>0.0, Versicolor=>0.0, Virginica=>1.0)

Vemos que a saída é a probabilidade do elemento pertencer a cada uma das classes. Podemos pegar a classe com a maior probabilidade usando a função __mode__ ou já aplicando na predição com __predict_mode__.

In [42]:
mode(y_pred[1])

CategoricalString{UInt32} "Virginica"

In [43]:
y_pred_mode = predict_mode(arvore, rows = teste)
@show y_pred_mode[1]

y_pred_mode[1] = "Virginica"


CategoricalString{UInt32} "Virginica"

Podemos agora medir a performance do nosso classificador, a biblioteca MLJ oferece algumas medidas, algumas destas são:
  - area_under_curve;
  - accuracy;
  - balanced_accuracy;
  - BrierScore;
  - cross_entropy;
  - FScore;
  - false_discovery_rate;
  - false_negative;
  - false_negative_rate;
  - false_positive;
  - false_positive_rate;
  - l1;
  - l2;
  - mae;
  - matthews_correlation;
  - misclassification_rate;
  - negative_predictive_value;
  - positive_predictive_value;
  - rms;
  - rmsl;
  - rmslp1;
  - rmsp;
  - true_negative;
  - true_negative_rate;
  - true_positive;
  - true_positive_rate;
  
Mais detalhes sobre as medidasde deempenho podem ser obtida na [documentação.](https://alan-turing-institute.github.io/MLJ.jl/stable/performance_measures/)

Podemos fazer uso da função __measures__ para ver todas as funções disponívies.





In [84]:
for m in measures()
    println(rpad(m.name, 29), m.orientation)
end

area_under_curve             score
accuracy                     score
balanced_accuracy            score
cross_entropy                loss
FScore                       score
false_discovery_rate         loss
false_negative               loss
false_negative_rate          loss
false_positive               loss
false_positive_rate          loss
l1                           loss
l2                           loss
mae                          loss
matthews_correlation         score
misclassification_rate       loss
negative_predictive_value    score
positive_predictive_value    score
rms                          loss
rmsl                         loss
rmslp1                       loss
rmsp                         loss
true_negative                score
true_negative_rate           score
true_positive                score
true_positive_rate           score
BrierScore{UnivariateFinite} score
DWDMarginLoss()              loss
ExpLoss()                    loss
L1HingeLoss()                loss
L2


Vamos separar somente as funções utilizadas para verificar o score de um modelo.

In [87]:
for m in measures()
    if m.orientation == :score
        println(rpad(m.name, 29))
    end
end

area_under_curve             
accuracy                     
balanced_accuracy            
FScore                       
matthews_correlation         
negative_predictive_value    
positive_predictive_value    
true_negative                
true_negative_rate           
true_positive                
true_positive_rate           
BrierScore{UnivariateFinite} 


Podemos então utilizar algumas destas para verificar o desempenho do nosso modelo.

In [56]:
accuracy(y_pred_mode, y[teste])

0.9555555555555556

In [75]:
balanced_accuracy(y_pred_mode, y[teste])

0.9576923076923077

Podemos também faze a matriz de confusão do nosso modelo:

In [101]:
confusion_matrix(y_pred_mode, y[teste])

┌ Warning: The classes are un-ordered,
│ using order: ["Setosa", "Versicolor", "Virginica"].
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase /home/phelipe/.julia/packages/MLJBase/uJ1jK/src/measures/confusion_matrix.jl:87


              ┌─────────────────────────────────────────┐
              │              Ground Truth               │
┌─────────────┼─────────────┬─────────────┬─────────────┤
│  Predicted  │   Setosa    │  Versicol…  │  Virginica  │
├─────────────┼─────────────┼─────────────┼─────────────┤
│   Setosa    │     12      │      0      │      0      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│  Versicol…  │      0      │     12      │      1      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│  Virginica  │      0      │      1      │     19      │
└─────────────┴─────────────┴─────────────┴─────────────┘


### Modelo não supervisionado

Agora faremos o uso de um modelo não supervisionado

In [89]:
v = [1, 2, 3, 4]
stand_model = UnivariateStandardizer()
stand = machine(stand_model, v)

Machine{UnivariateStandardizer} @ 1…95


In [90]:
fit!(stand)

┌ Info: Training Machine{UnivariateStandardizer} @ 1…95.
└ @ MLJBase /home/phelipe/.julia/packages/MLJBase/uJ1jK/src/machines.jl:182


Machine{UnivariateStandardizer} @ 1…95


Modelos não supervisionados definem um método __transform__, e podem implementar opcionalmente um método __inverse_transform__ . Assim como no caso supervisionado, aqui também fizemos o uso de uma __machine__ para guardar o nosso modelo e nosso conjunto de dados.

In [91]:
w = transform(stand)

4-element Array{Float64,1}:
 -1.161895003862225
 -0.3872983346207417
  0.3872983346207417
  1.161895003862225

In [93]:
@show round.(w, digits=2)
@show mean(w)
@show std(w);

round.(w, digits = 2) = [-1.16, -0.39, 0.39, 1.16]
mean(w) = 0.0
std(w) = 1.0


Neste caso nosso modelo tem uma transformação inversa:

In [95]:
vv = inverse_transform(stand, w)
sum(abs.(vv .- v))

0.0